## Load and parse PDFs

# Section 1: Setup and Dependencies

In [ ]:
!pip install -qU llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git
!pip install -qU llama-index
!pip install transformers accelerate bitsandbytes
!pip install llama-index-llms-huggingface
!pip install llama_index.llms.openai
!pip install llama-index-embeddings-huggingface
import torch

provide a API keys to continue

In [ ]:
import os
import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")
os.environ["HF_TOKEN"] = getpass.getpass("HF API Key:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Open AI Key:")

Import

In [ ]:
# Make sure we can run async in our Colab instance
import nest_asyncio

nest_asyncio.apply()

In [ ]:
#result_type - at time of writing this notebook the options are limited to "text" and "markdown".
#Markdown will be our choice as it will retain structured information quite nicely.
#num_workers - this will let us set how many workers we'll need.
#Generally we'll want to set this to the number of files we're going to need to parse. (the maximum is 10)
# NOTE: As of time of writing, only .pdf files are accepted

from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="en",
    num_workers=2,
)

In [ ]:
from llama_index.core import SimpleDirectoryReader
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    "/content/PDF", file_extractor=file_extractor
).load_data()

In [ ]:
# example of parsed document, is readable in Markdown
# check how it looks on the markdown.preview website
print(documents[0].text[:1000])

# Section 2: Creating Embeddings and Models

In [ ]:
# Settings of the model and. embeddings type
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate



quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-alpha",
    tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.0, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
# Download the markdown parser
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8)

In [ ]:
# let's parse
#nodes = node_parser.get_nodes_from_documents(documents)
nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
# Now we can extract our `base_nodes` and `objects` to create our `VectorStoreIndex`
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)


In [ ]:
# Let's build the index!
from llama_index.core import VectorStoreIndex

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

# Section 3: Saving and Loading the Database


[See this article](https://medium.com/@reddyyashu20/llamaindex-create-save-load-indexes-customize-llms-prompts-embeddings-abb581df6dac)

In [ ]:
save_dir="./myvector_store/newindex"
# create and load the index
# Persist index to disk
recursive_index.storage_context.persist(persist_dir=save_dir)

the vector store is located here:
Google drive

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
#LLAMA API
#HF API
#LLM model
#embed model
# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="/content/vectors")

# Load index from the storage context
new_index = load_index_from_storage(storage_context)


# Let's do a simple test
# Create a query engine from the index
new_query_engine = new_index.as_query_engine()
response = new_query_engine.query("why should i choose MAN?")
print(response)

## Recursive Query Engine

Now we can build our Recursive Query Engine with reranking

In [ ]:
import locale
# Storing the original function
original_getpreferredencoding = locale.getpreferredencoding

# Restoring the original function
locale.getpreferredencoding = original_getpreferredencoding

In [ ]:
# Modifying the lambda to accept an unused argument
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=15,
    node_postprocessors=[reranker],
    verbose=True
)

In [ ]:
query = "which department should contact for issues related to Capacity content?"
response = recursive_query_engine.query(query)

In [ ]:
print(response)

In [ ]:
query = "who are the contact persons for the Supply Subject matter of the content?"
response = recursive_query_engine.query(query)

In [ ]:
print(response)

In [ ]:
query = "How do I get access to FIN?"
response = recursive_query_engine.query(query)

In [ ]:
print(response)

In [ ]:
query = "How to check your vendor number?"
response = recursive_query_engine.query(query)
print(response)

Answer: The number is given on the documents you receive from MAN: payment advice, debit note, order, request for payment, etc. In case of problems with determining the number, please contact vendor.support@example.com


In [ ]:
query = "How to check the debit note number?"
response = recursive_query_engine.query(query)
print(response)

Answer: The number is included on the document sent to your company. It is a 10-digit number (FI-Belegnummer). Depending on the MAN company, the number of the debit note starts with the numbers shown below:
MAN Truck & Bus SE, Germany (Munich, Salzgitter, Dachau, Nuernberg) ...

In [ ]:
query = "I hope this email finds you well. As one of our valued partners in the automotive industry, we strive to ensure seamless communication and collaboration between our companies. In this regard, we would like to inquire about the process for verifying our debit note  number with your esteemed company. Could you please provide guidance on how to check our vendor number within your system? Ensuring the accuracy of our vendor information is crucial for smooth transactions and efficient business operations. Your assistance in this matter would be greatly appreciated. If there are any specific documents or procedures we need to follow, please do not hesitate to inform us. Thank you for your attention to this matter. We look forward to your prompt response. Best regards,Sam"
response = recursive_query_engine.query(query)
print(response)

"Could you please provide guidance on how to check our vendor number within your system?"

```
# This is formatted as code
```



In [ ]:
query = "Could you please provide guidance on how to check our vendor number within your system?"
response = recursive_query_engine.query(query)
print(response)

In [ ]:
query = "Where the cost center G8033 is located?"
response = recursive_query_engine.query(query)
print(response)